In [12]:
!python -m pip install --user py2neo


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from py2neo import Graph
# Connect to the Neo4j Database
graph = Graph("bolt://localhost:7687", user="neo4j", password="asd123456")

In [ ]:
# Reset database (For error correction)
graph.run("""
MATCH (n:Company)
DETACH DELETE n
""")

(No data)

In [7]:
# Import Company Node
graph.run("""
LOAD CSV FROM 'file:///hidy.nodes.company.csv' AS row
WITH row
SKIP 1
CREATE (:Company {id: row[0], name: row[1], code: row[2]})
""")

(No data)

In [8]:
# Import relationships
relationships = ['compete', 'cooperate', 'dispute', 'invest', 'same_industry', 'supply']
for rel in relationships:
    query = """
    LOAD CSV FROM 'file:///hidy.relationships.{rel}.csv' AS row
    WITH row
    SKIP 1
    MATCH (start:Company {{id: row[0]}})
    MATCH (end:Company {{id: row[1]}})
    MERGE (start)-[r:{rel_type}]->(end)
    """.format(rel=rel, rel_type=rel.upper())
    graph.run(query)

In [11]:
# Query the number of nodes and relationships
nodes_count = graph.run("MATCH (n:Company) RETURN count(n) AS count").data()
print(f"Number of nodes: {nodes_count[0]['count']}")

for rel in relationships:
    rel_count = graph.run(f"MATCH ()-[:{rel.upper()}]->() RETURN count(*) AS count").data()
    print(f"Number of {rel} relationships: {rel_count[0]['count']}")


Number of nodes: 3974
Number of compete relationships: 25
Number of cooperate relationships: 3402
Number of dispute relationships: 427
Number of invest relationships: 542
Number of same_industry relationships: 5273
Number of supply relationships: 1375
